## $\textbf{Task1: RSA Decryption and Message Translation }$

This task involves decrypting a message that has been encrypted using the RSA encryption algorithm. In RSA, the message is encrypted with a public key and can only be decrypted using a private key. Our goal is to:

1. **Decrypt** the message (which is in the form of numbers).
2. **Translate** the decrypted numbers into readable text using a custom character mapping.

### RSA Encryption and Decryption Overview

In RSA encryption:
- The message is converted into a number, let say `M`.
- This number is encrypted using the formula:
  $$ C \equiv M^e \mod n $$
  Where:
  - `C` is the ciphertext (the encrypted message).
  - `M` is the plaintext (the original message).
  - `e` is the public exponent.
  - `n` is the product of two large prime numbers, `p` and `q`.

Decryption is done using the private key:
- The private key `d` is calculated such that:
  $$ e \cdot d \equiv 1 \mod \phi(n) $$
  Where $`\phi(n)`$ is Euler’s totient function for `n`, and `d` is the private exponent. The decrypted message is obtained by:
  $$ M \equiv C^d \mod n $$
  Where `C` is the ciphertext and `M` is the original message.

### Problem Breakdown

- **Input**: We are given the encrypted message `M` (in the form of a list of large numbers), public key values `n` and `e`.
- **Output**: The decrypted message is converted back to a string of characters based on a custom character mapping.
- **Goal**: Decrypt the message, then map the resulting numbers to letters or spaces as defined by a given mapping.
---


In [ ]:
from sympy import factorint # For factorize n into p and q
from math import gcd

# Problem data
n = 956331992007843552652604425031376690367
e = 12398737
M = [427849968240759007228494978639775081809,
     498308250136673589542748543030806629941,
     925288105342943743271024837479707225255,
     95024328800414254907217356783906225740]

In [ ]:
# Factorizing n to get p and q
factors = factorint(n)
p, q = list(factors.keys())
print(f"p = {p}, q = {q}")

p = 7746289204980135457, q = 123456789012345681631


In [ ]:
# Calculating Euler's totient function phi(n)
phi_n = (p - 1) * (q - 1)
print(f"Phi(n) = {phi_n}")

Phi(n) = 956331992007843552521401346814050873280


In [ ]:
# Finding d such that e * d ≡ 1 (mod phi(n))
def extended_gcd(a, b):
    """
    This function computes the greatest common divisor (GCD) of a and b
    using the extended Euclidean algorithm, which also returns the
    coefficients for the equation: a * x + b * y = gcd(a, b).
    """
    if b == 0:
        return a, 1, 0
    g, x1, y1 = extended_gcd(b, a % b)
    x = y1
    y = x1 - (a // b) * y1
    return g, x, y # where g is the gcd

In [ ]:
def mod_inverse(e, phi_n):
    """
    This function returns the modular inverse of e modulo phi_n.
    That is, it finds d such that (e * d) % phi_n = 1.
    """
    g, x, _ = extended_gcd(e, phi_n)
    if g != 1:
        raise Exception("Modular inverse does not exist")
    return x % phi_n

d = mod_inverse(e, phi_n)
print(f"Private exponent d = {d}")

Private exponent d = 801756262003467870842260800571951669873


In [ ]:
# Decrypting the message M
M_decrypted = [pow(m, d, n) for m in M]
print(f"Decrypted message (as numbers): {M_decrypted}")

Decrypted message (as numbers): [30181929001929002335002215303015280030, 25003018150033252822140030181130002415, 32152800332825301500302500231500152319, 223500141913211924292524]


In [ ]:
# Create the mapping (numbers to letters)
code_mapping = {"00": " ", **{str(i+11): chr(65+i) for i in range(26)}}

# Decode each number
msg_list = []
for num in M_decrypted:
    num_txt = str(num)

    if len(num_txt) % 2 != 0:
        num_txt = "0" + num_txt

    msg_list.append([code_mapping.get(num_txt[i:i+2], "?") for i in range(0, len(num_txt), 2)])

In [ ]:
# Print the message decoded
for msg in msg_list:
    print("".join(msg))

THIS IS MY LETTER T
O THE WORLD THAT NE
VER WROTE TO ME EMI
LY DICKINSON


In [ ]:
# Print the full message
print("Full message decoded:", "".join("".join(msg) for msg in msg_list))

Full message decoded: THIS IS MY LETTER TO THE WORLD THAT NEVER WROTE TO ME EMILY DICKINSON


## $\textbf{Task2: RSA Cryptosystem with 600-Digit Safe Primes }$

In this task, we will implement an RSA cryptosystem using two **safe prime numbers** with 600 digits. A safe prime is a prime number `p` where `(p - 1) / 2` is also a prime.

The process involves:
1. **Generating two 600-digit safe primes** `p` and `q`. 600 digits is about 1992 bits.
2. **Checking the primality of `p` and `q`**
3. **Computing the modulus `n`** as the product of `p` and `q`, and **calculating Euler's totient function** `φ(n)` as seen in exo2.
4. **Generating the public and private keys** for the RSA system.
5. **Encrypting a message** using the public key.
6. **Decrypting the message** using the private key.

### RSA Key Generation
RSA relies on the following steps to generate the public and private keys:
- The modulus `n` is the product of two prime numbers `p` and `q`.
- The public exponent `e` is usually a small value like 65537.
- The private exponent `d` is the modular inverse of `e` modulo Euler’s totient `phi(n)`.

### Key Components of RSA

1. **Public Key**: `(n, e)`
2. **Private Key**: `(n, d)`

### Message Encryption and Decryption

The RSA encryption and decryption are performed (As seen in Task1) using the formulas:
- Encryption: $ C \equiv M^e mod n $
- Decryption: $ M \equiv C^d mod n $

Where `C` is the ciphertext, `M` is the plaintext, `e` is the public exponent, `d` is the private exponent, and `n` is the modulus.

**Note**: In this task, the functions will be built using custom tests like `strong_pseudoprimality_test` instead of Python's built-in functions like `isprime`, which may lead to slightly slower execution times.

---

In [ ]:
import random
import math

def strong_pseudoprimality_test(N, t=10):
    """
    Strong pseudoprimality test for an odd integer N >= 3.
    Repeats the test t times independently.

    Returns "composite" if N is composite, otherwise "probably prime".
    """
    if N < 3 or N % 2 == 0:
        return "composite"

    # Write N-1 as 2^e * m, with m odd
    e = 0
    m = N - 1
    while m % 2 == 0:
        m //= 2
        e += 1

    for _ in range(t):
        # Choose random x in {2, ..., N-2}
        x = random.randint(2, N - 2)

        if gcd(x, N) != 1:
            return "composite"

        # Compute y = x^m mod N
        y = pow(x, m, N)

        if y == 1 or y == N - 1:
            continue

        # Iterate to check for -1 condition
        for _ in range(e - 1):
            y = pow(y, 2, N)
            if y == N - 1:
                break
        else:
            return "composite"

    return "probably prime"

print(strong_pseudoprimality_test(11))
print(strong_pseudoprimality_test(561))
print(strong_pseudoprimality_test(7))
print(strong_pseudoprimality_test(101))

probably prime
composite
probably prime
probably prime


In [ ]:
import sympy

# Generate safe prime number function
def generate_safe_prime(bit_length):
    """
    Generates a safe prime p, where p = 2q + 1 and q is also prime,
    using the strong pseudoprimality test.
    """
    x = 2 ** bit_length
    ln2x = sympy.log(x, 2)

    # Bounds y0 et y1
    y0 = (x - ln2x) / (2 * ln2x)
    y1 = 2**(bit_length-1) - 1

    while True:

        # Research q in [y0, y1]
        q = random.randint(int(y0), int(y1))
        if strong_pseudoprimality_test(q) == "probably prime" and strong_pseudoprimality_test(2 * q + 1) == "probably prime":
            return 2 * q + 1

    # Search for powers of 2
    for j in range(bit_length // 4, bit_length // 2):
        q1 = random.randint(2**j, 2**(j+1))
        q2 = random.randint(int(y0 / q1), int(y1 / q1))

        # Check primality of q1, q2, and 2q2 + 1
        if strong_pseudoprimality_test(q1) == "probably prime" and strong_pseudoprimality_test(q2) == "probably prime" and strong_pseudoprimality_test(2 * q2 + 1) == "probably prime":
            return 2 * q2 + 1

# Test
bit_length = 500
safe_prime = generate_safe_prime(bit_length)
print(f"Generated safe prime p = {safe_prime}")

Generated safe prime p = 2707805907492098808865786847904786463678869184944572920971132832427761021148815872370707740932609855057791009748926260197262551396623348832111521809963


In [ ]:
# Check if p is a safe prime number
def is_safe_prime(p):
    """
    Verifies if p is a safe prime, i.e., p = 2q + 1 and q is prime.
    Returns True if p is a safe prime, False otherwise.
    """
    if p % 2 == 0 or p < 3:
        return False

    # Check if p = 2q + 1 for some prime q
    q = (p - 1) // 2
    if 2 * q + 1 != p:
        return False

    # Check if q is prime
    if strong_pseudoprimality_test(q) != "probably prime":
        return False

    return True

if is_safe_prime(safe_prime):
    print(f"Its a valid safe prime.")
else:
    print(f"Its is NOT a valid safe prime.")

Its a valid safe prime.


In [ ]:
# RSA key generation Function
def generate_rsa_keys(bit_length):

    p = generate_safe_prime(bit_length)
    q = generate_safe_prime(bit_length)

    while q == p:
        q = generate_safe_prime(bit_length)

    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537

    if gcd(e, phi) != 1:
        raise Exception("e is not coprime with phi(n); regenerate the keys.")

    d = mod_inverse(e, phi)

    return (n, e, d, p, q)

In [ ]:
# Encryption function using `C = M^e mod n`
def encrypt(msg, public_key):
    """
    Encrypt the msg using RSA encryption with the public key (e, n).
    """
    e, n = public_key
    msg_int = int(msg)
    return pow(msg_int, e, n)

In [ ]:
# Decryption function: `M = C^d mod n`
def decrypt(encrypted_msg, private_key):
    """
    Decrypt the encrypted msg using RSA decryption with the private key (d, n).
    """
    d, n = private_key
    return pow(encrypted_msg, d, n)

**Note to the tutor/examiner for the cell below:**  
<span style="color:red">Verify the result first before running the cell below. It takes a long time to run!</span>

In [ ]:
# Generate RSA keys
bit_length = 1992
n, e, d, p, q = generate_rsa_keys(bit_length)

# Define public and private keys
public_key = (e, n)
private_key = (d, n)

print(f"RSA keys generated:\nPublic key: {public_key}\nPrivate key: {private_key}")

RSA keys generated:
Public key: (65537, 220759290840871014751375903446479257956984477745245099220993973152987230173326860306190674273252034305055156493068087707118339655006008442370304840448499650702421588581962059834807817578468404208637318024362758509704211246527767841098847490567587585919036077587768969810548276784997492698553501284006802551450998063029013979657167292570923909605193103944918010836300788609434382058285925123133490354393450587400308304056802383282909111931311543841656951313852405343672213405798886068020158850101444589707885505412739220471842183141609733854697380349596133143531129317833064746125165864573054606182390139414506993094409356092931362331771376456464939029329678185497937283295433944909911285520010569727926366327008766594648166976643562640127107368887886770543412091510597823518882371765568852716698013086329044685431707153596984134846549855251612669516845449374417482699264107578790443333227486621425377709006148594232172412241621397291120838429968656204828959511

In [ ]:
# Encoding and encrypting the msg
encoded_msg = "18152222250019230029112215350011002930311415243000113000111923290028331124141100112414001124001129261928192417001411301100291319152430192930"
encrypted_msg = encrypt(encoded_msg, public_key)
print(f"Encrypted msg: {encrypted_msg}")

# Decrypt the msg using the private key
decrypted_integer = decrypt(encrypted_msg, private_key)
print(f"Decrypted msg: {decrypted_integer}")

Encrypted msg: 1751479779487287529143792069856008179561652618184831432834388067477313240624883845930190305116167643515597135950363444368597168251446330605924335392173212525584203308790593140893198419301362211569630966458056260670144226427148957450808016689665799588385524039011379119574354693504130086841600148829430744948875282357791234406087448480164872735369683357194378757794812586404291376483501396253335127087346365253982433578354819954816303577330090571445122334646671903125508514324374610516284091116544147127286357699291164629390418337850676413126421671641698603729369209393135080284426878462935642688730762731347043328192080351248992024639867582105907952229093735254451627699426981080096314662277045932728596058485011374942827503232421912394321735233809861682290534797494457949537392333047810317654364047484343793896775503254421936150969310759652766705289593019411615432126217562380003728136538277739255656949094610923253643560848337626642279152649805402751589502648075490778032526943571387

In [ ]:
# Convert decrypted integer into a string
m_str = str(decrypted_integer)

if len(m_str) % 2 != 0:
    m_str = "0" + m_str

# Decoding the msg using the mapping (code_mapping in Exo 2)
msg_clear = ""
for i in range(0, len(m_str), 2):
    pair = m_str[i:i + 2]
    if pair in code_mapping:
        msg_clear += code_mapping[pair]

# Output the decoded msg
print(f"Decoded msg: {msg_clear}")

Decoded msg: HELLO IM SALEY A STUDENT AT AIMS RWANDA AND AN ASPIRING DATA SCIENTIST
